<font style='font-weight:750;font-size:16px;text-decoration:underline;'>Imports</font>

In [1]:
import pandas as pd 
import numpy as np
from numpy.linalg import solve # in order to solve matrix equations
import matplotlib.pyplot as plt

<font style='font-weight:750;font-size:16px;text-decoration:underline;'>Functions</font>

In [551]:
# performing alternating least squares
# def als(R_, U_, I_, K_, mu, bias_user, bias_item, lambda_user_, lambda_item_, lambda_bu, lambda_bi ): ==> model with biases (not used eventually)
def als(R_, U_, I_, K_, lambda_user_, lambda_item_):
    flag=0
    identity_matrix = np.eye(K_)
    #update the users biases

        
    #update user latent vectors
    for user, ratings in enumerate(R_):
        if user == 0: continue
        # count the number of item of the user
        num_of_items =  np.count_nonzero(ratings)
        # in order to avoid singular matrix
        if (num_of_items == 0): num_of_items = 1
        # extracting the items ids of the the user
        items_of_user = np.nonzero(ratings)[0]
        # extracting the fixed vectors of the items of the user
        I_vectors = I_[items_of_user,: ]
        # extracting the ratings of the items of the user
        # ratings_of_user = R_[user, items_of_user]-(mu+bias_user[user])*np.ones([num_of_items])-bias_item[items_of_user]==> model with biases (not used eventually)
        ratings_of_user = R_[user, items_of_user]
        a = I_vectors.T.dot(I_vectors) + lambda_user_* num_of_items * identity_matrix
        b= ratings_of_user.dot(I_vectors)
        U_[user,: ] = solve(a,b)

    #update item latent vectors
    for item, ratings in enumerate(R_.T):
        if item==0:continue
        # count the number of users of the item
        num_of_users =  np.count_nonzero(ratings)
        # in order to avoid singular matrix
        if (num_of_users == 0): continue
        # extracting the users ids of the the item
        users_of_item = np.nonzero(ratings)[0]
        # extracting the fixed vectors of the users of the item
        U_vectors = U_[users_of_item,:]
        #ratings_of_item = R_[users_of_item, item]-(mu+bias_item[item])*np.ones([num_of_users])-bias_user[users_of_item]==> model with biases (not used eventually)
        ratings_of_item = R_[users_of_item, item]
        a = U_vectors.T.dot(U_vectors) + lambda_item_* num_of_users * identity_matrix
        b= ratings_of_item.dot(U_vectors)
        I_[item,:] =solve(a,b)
    
  #updating User biases ==> model with biases (not used eventually)       
#     for user, ratings_items in enumerate(R_):
#         # User 0 doesn't exists
#         if user == 0: continue  
#         #count the numbers of items rated by the specific user
#         num_of_items =  np.count_nonzero(ratings_items)
#         items_of_user = np.nonzero(ratings_items)[0]
#         #to avoid dividing by 0
#         if num_of_items==0: num_of_items=1
#         a = (1/(num_of_items+ lambda_bu))
#         b = 0     
#         for item in items_of_user:
#             b += R_[user,item]-mu-bias_item[item]-np.matmul(U_[user,:].T,I_[item,:])
#         bias_user[user] =a*b
        
    #update the items biases==> model with biases (not used eventually)
#     for item, rated_users in enumerate(R_.T):
#         # item 0 doesn't exist
#         if item == 0: continue  
#         #count the numbers of users who rated the specific item
#         num_of_users =  np.count_nonzero(rated_users)
#         users_of_item = np.nonzero(rated_users)[0]
#         if num_of_users==0: num_of_users=1
#         a = (1/(num_of_users+lambda_bi))
#         b = 0     
#         for user in users_of_item:
#             b += (R_[user,item]-mu-bias_user[user]-np.matmul(U_[user,:].T,I_[item,:]))
#         bias_item[item] =a*b
#     return U_, I_, bias_user, bias_item

    return U_, I_

In [552]:
# bulit the matrix rating from U - user matrix and I - items matrix
def mat_builder(U_, I_):
    return U_.dot(I_.T)

In [553]:
def get_error(actual_mat, prediction_mat, error_type='rmse'):
    mat_boolean = (actual_mat > 0)
    m = np.mean(actual_mat[np.where(actual_mat != 0)])
    erorr = 0
    if error_type == 'rmse':
        error = np.sqrt(np.sum(mat_boolean * pow(actual_mat - prediction_mat, 2)) / np.sum(mat_boolean))
    elif error_type == 'mse':
        error = np.sum(mat_boolean * pow(actual_mat - prediction_mat, 2)) / np.sum(mat_boolean)
    elif error_type == 'mae':
        error = np.sum(mat_boolean * abs(actual_mat - prediction_mat)) / np.sum(mat_boolean)
    elif error_type == 'r_sq':
        error = 1-(np.sum(mat_boolean * pow(actual_mat - prediction_mat, 2))/np.sum(mat_boolean * pow(actual_mat - m, 2)))
    else:
        print('{} is not supprted, please set one of the following as the error metric:\\nrmse, mse, mae, r_sq'.format(error_type))
    return error

In [554]:
# performing random search in order to find the best hyper-parameters
def evaluate(train_R_, val_R_, ):   
    best_model = {}
    lowest_erorr = 5
    evaluations = 20
    # in each evaluation we are genarting difrent set of hyper-parameters and train the model with them
    for eval in range(evaluations):
        print("*******************************************************************************************")
        print('eval #{}'.format(eval+1))
        K = int(np.random.uniform(5, 40))
        lambda_user = np.random.uniform(0.001, 0.1)
        lambda_item = np.random.uniform(0.001, 0.1) 
        #lambda_bu = np.random.uniform(0.001, 0.1)==> model with biases (not used eventually)
        #lambda_bi = np.random.uniform(0.001, 0.1)==> model with biases (not used eventually)
        U = np.random.randn(int(train_R_.shape[0]), K)
        I = np.random.randn(int(train_R_.shape[1]), K) 
        #creating initial biases ==> model with biases (not used eventually)
        #bias_U= np.zeros([users_index])==> model with biases (not used eventually)
        #bias_U[:]=np.random.uniform(-0.1, 0.1)==> model with biases (not used eventually)
        #bias_I= np.zeros([items_index])==> model with biases (not used eventually)
        #bias_I[:]=np.random.uniform(-0.1, 0.1)==> model with biases (not used eventually)
        training_process = []
        val_process = []
        iterations = 30
        for iter in range(iterations):
            #U, I, bias_U, bias_I  = als(train_R_, U, I, K, mu_train,bias_U,bias_I,lambda_user,lambda_item,lambda_bu,lambda_bi)==> model with biases (not used eventually)
            U, I = als(train_R_, U, I, K,lambda_user,lambda_item)
            prediction = mat_builder(U, I)
            error = get_error(train_R_, prediction,'rmse')
            val_error = get_error(val_R_, prediction,'rmse')
            training_process.append(error)
            val_process.append(val_error)
            print("Iteration: %d ; error = %.4f; val_error = %.4f" % (iter+1, error, val_error))
            if iter > 1 and abs(error - training_process[iter-2]) <0.001: break
        # checking if we have found a better model 
        if val_process[-1] < lowest_erorr: 
            lowest_erorr = val_process[-1]
            best_model['K'] = K
            best_model['lambda_user'] = lambda_user
            best_model['lambda_item'] = lambda_item
            best_model['rmse'] = lowest_erorr
            best_model['mse'] = get_error(val_R_, prediction,'mse')
            best_model['mae'] = get_error(val_R_, prediction,'mae')
            best_model['r_sq'] = get_error(val_R_, prediction,'r_sq')
            print('the bast model is:\\n', best_model)
    return best_model['K'], best_model['lambda_user'], best_model['lambda_item']

In [555]:
#load the data\n",
df_train = pd.read_csv("Train.csv")
#df_train = df_train_real[(df_train_real['User_ID_Alias']<= 2000) & (df_train_real['Movie_ID_Alias']<= 2000 )]
df_val = pd.read_csv("Validation.csv")
#df_val = df_val_real[(df_val_real['User_ID_Alias']<= 2000) & (df_val_real['Movie_ID_Alias']<= 2000)]
df_train_val = pd.concat([df_train_real, df_val])
df_test = pd.read_csv("Test.csv")
#df_test = df_test_real[(df_test_real['User_ID_Alias']<= 2000) & (df_val_real['Movie_ID_Alias']<= 2000)]

In [556]:
#create the R matrixes of train, val and train_val
users_index=int(df_train_val['User_ID_Alias'].max()) + 1
items_index=int(df_train_val['Movie_ID_Alias'].max()) + 1
#create the R matrixes of train, val and train_val
train_R =np.zeros([users_index, items_index])
# sum_train_ratings = 0 (for "mu" clac- not used eventually==> model with biases)
# count = 0 (for "mu" clac- not used eventually==> model with biases)
for i, record in df_train.iterrows():
    row = record
    user_id = int(row['User_ID_Alias'])
    item_id = int(row['Movie_ID_Alias'])
    rating = int(row['Ratings_Rating'])
    train_R[user_id, item_id] = rating
    #sum_train_ratings+=rating (for "mu" clac- not used eventually==> model with biases)
    #count+=1 (for "mu" clac- not used eventually==> model with biases)
# mu_train = sum_train_ratings/count (for "mu" clac- not used eventually==> model with biases)
val_R =np.zeros([users_index, items_index])
for i, record in df_val.iterrows():
    row = record
    user_id = int(row['User_ID_Alias'])
    item_id = int(row['Movie_ID_Alias'])
    rating = int(row['Ratings_Rating'])
    val_R[user_id, item_id] = rating
    train_val_R =np.zeros([users_index, items_index])
for i, record in df_train_val.iterrows():
    row = record
    user_id = int(row['User_ID_Alias'])
    item_id = int(row['Movie_ID_Alias'])
    rating = int(row['Ratings_Rating'])
    train_val_R[user_id, item_id] = rating
    # sum_train_val_ratings+=rating(for "mu" clac- not used eventually==> model with biases)
    # count+=1 (for "mu" clac- not used eventually)
# mu_train_val = sum_train_val_ratings/count (for "mu" clac- not used eventually==> model with biases)

In [557]:
# getting the best hyper-parameters
final_K, final_lambda_user, final_lambda_item  = evaluate(train_R, val_R)

*******************************************************************************************
eval #1
Iteration: 1 ; error = 2.1412; val_error = 2.8777
Iteration: 2 ; error = 0.8900; val_error = 0.9899
Iteration: 3 ; error = 0.8223; val_error = 0.9363
Iteration: 4 ; error = 0.8003; val_error = 0.9240
Iteration: 5 ; error = 0.7879; val_error = 0.9174
Iteration: 6 ; error = 0.7798; val_error = 0.9128
Iteration: 7 ; error = 0.7742; val_error = 0.9095
Iteration: 8 ; error = 0.7702; val_error = 0.9069
Iteration: 9 ; error = 0.7672; val_error = 0.9049
Iteration: 10 ; error = 0.7649; val_error = 0.9033
Iteration: 11 ; error = 0.7631; val_error = 0.9019
Iteration: 12 ; error = 0.7617; val_error = 0.9009
Iteration: 13 ; error = 0.7606; val_error = 0.8999
Iteration: 14 ; error = 0.7596; val_error = 0.8992
Iteration: 15 ; error = 0.7588; val_error = 0.8985
Iteration: 16 ; error = 0.7582; val_error = 0.8979
Iteration: 17 ; error = 0.7576; val_error = 0.8973
Iteration: 18 ; error = 0.7571; val_error 

Iteration: 1 ; error = 2.2133; val_error = 3.6261
Iteration: 2 ; error = 0.8350; val_error = 1.0119
Iteration: 3 ; error = 0.7403; val_error = 0.9461
Iteration: 4 ; error = 0.7185; val_error = 0.9342
Iteration: 5 ; error = 0.7087; val_error = 0.9304
Iteration: 6 ; error = 0.7032; val_error = 0.9289
Iteration: 7 ; error = 0.6997; val_error = 0.9278
Iteration: 8 ; error = 0.6973; val_error = 0.9270
Iteration: 9 ; error = 0.6955; val_error = 0.9265
Iteration: 10 ; error = 0.6941; val_error = 0.9262
Iteration: 11 ; error = 0.6930; val_error = 0.9260
Iteration: 12 ; error = 0.6921; val_error = 0.9259
Iteration: 13 ; error = 0.6914; val_error = 0.9259
Iteration: 14 ; error = 0.6907; val_error = 0.9258
Iteration: 15 ; error = 0.6902; val_error = 0.9258
Iteration: 16 ; error = 0.6897; val_error = 0.9259
Iteration: 17 ; error = 0.6893; val_error = 0.9260
*******************************************************************************************
eval #10
Iteration: 1 ; error = 1.8370; val_error 

Iteration: 3 ; error = 0.6928; val_error = 1.1153
Iteration: 4 ; error = 0.6656; val_error = 1.1152
Iteration: 5 ; error = 0.6518; val_error = 1.1182
Iteration: 6 ; error = 0.6433; val_error = 1.1178
Iteration: 7 ; error = 0.6375; val_error = 1.1141
Iteration: 8 ; error = 0.6333; val_error = 1.1108
Iteration: 9 ; error = 0.6302; val_error = 1.1085
Iteration: 10 ; error = 0.6277; val_error = 1.1066
Iteration: 11 ; error = 0.6257; val_error = 1.1053
Iteration: 12 ; error = 0.6241; val_error = 1.1044
Iteration: 13 ; error = 0.6227; val_error = 1.1041
Iteration: 14 ; error = 0.6215; val_error = 1.1043
Iteration: 15 ; error = 0.6205; val_error = 1.1050
Iteration: 16 ; error = 0.6197; val_error = 1.1061
Iteration: 17 ; error = 0.6189; val_error = 1.1074
Iteration: 18 ; error = 0.6183; val_error = 1.1089
Iteration: 19 ; error = 0.6177; val_error = 1.1104
Iteration: 20 ; error = 0.6172; val_error = 1.1120
Iteration: 21 ; error = 0.6167; val_error = 1.1135
**************************************

In [560]:
#after the random search we have the final hyper-paramters
# now we want to train the model with the final hyper-paramters and predict the ratings for the test file
final_U = np.random.randn(users_index, final_K)
final_I = np.random.randn(items_index, final_K)
process = []
iterations = 30
for iter in range(iterations):
    # final_U, final_I, bias_u, bias_i, bias_user, bias_item = als(train_val_R , final_U, final_I, bias_u, bias_i, final_K, final_lambda_user, final_lambda_item, final_lambda_bu,final_lambda_bi)==> model with biases (not used eventually)
    final_U, final_I = als(train_val_R , final_U, final_I, final_K, final_lambda_user, final_lambda_item)
    prediction = mat_builder(final_U, final_I)
    error = get_error(train_val_R, prediction,'rmse')
    process.append(error)
    print("Iteration: %d ; error = %.4f" % (iter+1, error))
    if iter > 1 and abs(error - process[iter-2]) <0.001: break
        
results = {}
results['K'] = final_K
results['lambda_user'] = final_lambda_user
results['lambda_item'] = final_lambda_item
results['rmse'] = get_error(train_val_R, prediction,'rmse')
results['mse'] = get_error(train_val_R, prediction,'mse')
results['mae'] = get_error(train_val_R, prediction,'mae')
results['r_sq'] = get_error(train_val_R, prediction,'r_sq')
# results['MPR']=get_error(train_val_R,prediction,'mpr')
print('the final model is:\n')
print('K :', results['K'] ,'\n' )
print('Lambda user: ',results['lambda_user'] )
print('Lambda item: ',results['lambda_item'] )
print('RMSE: ',results['rmse'] )
print('MSE: ',results['mse'])
print('MAE: ',results['mae'])
print('R^2: ',results['r_sq'])

final_prediction = mat_builder(final_U, final_I)
df_test['Rating'] = 0
for i, record in df_test.iterrows():
    user_id = int(record['User_ID_Alias'])
    item_id = int(record['Movie_ID_Alias'])
    #if user_id < 2000 and item_id < 2000:
    rating = final_prediction[user_id, item_id]
    if rating >5: rating =5
    if rating <1: rating =1
    mask = (df_test['User_ID_Alias'] == user_id) & (df_test['Movie_ID_Alias'] == item_id)
    df_test.loc[mask, 'Rating'] = rating
df_test.to_csv (r'300339785_201314796_203016217.csv', index = None, header=True)

Iteration: 1 ; error = 2.3396
Iteration: 2 ; error = 0.9013
Iteration: 3 ; error = 0.8330
Iteration: 4 ; error = 0.8123
Iteration: 5 ; error = 0.8012
Iteration: 6 ; error = 0.7943
Iteration: 7 ; error = 0.7898
Iteration: 8 ; error = 0.7865
Iteration: 9 ; error = 0.7841
Iteration: 10 ; error = 0.7822
Iteration: 11 ; error = 0.7807
Iteration: 12 ; error = 0.7795
Iteration: 13 ; error = 0.7784
Iteration: 14 ; error = 0.7776
Iteration: 15 ; error = 0.7768
Iteration: 16 ; error = 0.7762
Iteration: 17 ; error = 0.7757
Iteration: 18 ; error = 0.7752
Iteration: 19 ; error = 0.7748
the final model is:

K : 14 

Lambda user:  0.0751674251827961
Lambda item:  0.07169854966403758
RMSE:  0.7747814500438175
MSE:  0.6002862953320005
MAE:  0.616939175314185
R^2:  0.5170806652048714


In [559]:
df_test.head(100)

,User_ID_Alias,Movie_ID_Alias,Rating
0,1,552,3.143570
1,2,1246,2.829510
2,3,1809,4.311538
3,4,716,4.202607
4,5,1475,3.737359
...,...,...,...
95,96,1878,3.362881
96,97,1809,3.505087
97,98,742,4.141690
98,99,942,1.646948
